In [1]:
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torchvision
from torch import nn
import numpy as np
import torch

In [2]:
transformation =transforms.Compose([
    transforms.ToTensor()  #转换到Tensor，并且转换为0-1之间,将channel 放到第一个纬度
])
train_ds = datasets.MNIST('data/',train = True,transform = transformation,download = True)
test_ds = datasets.MNIST('data/',train = False,transform = transformation,download = True)
# len(train_ds)
# len(test_ds)

1.7%

100.0%


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


100.0%


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw




6.0%

100.0%


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



100.0%

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [3]:
train_loader = DataLoader(train_ds,batch_size =64 ,shuffle = True,num_workers = 16)
test_loader = DataLoader(test_ds,batch_size =256 ,shuffle = False,num_workers = 16)

In [4]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(28*28,128)
        self.linear2 = nn.Linear(128,64)
        self.linear3 = nn.Linear(64,10)
    def forward(self,input):
        x = input.view(-1,28*28)
        x = nn.functional.relu(self.linear1(x))
        x = nn.functional.relu(self.linear2(x))
        y = self.linear3(x)
        return y

In [5]:
model = Model()
loss_fn  = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [6]:
def accuracy(y_pred,y_true):
    y_pred = torch.argmax(y_pred,dim=1)
    acc = (y_pred==y_true).float().mean()
    return acc
#测试集
def evaluate_testset(data_loader,model):
    acc_sum,loss_sum,total_example = 0.0,0.0,0
    for x,y in data_loader:
        y_hat = model(x)
        acc_sum += (y_hat.argmax(dim=1)==y).sum().item()
        loss = loss_fn(y_hat,y) 
        loss_sum += loss.item()
        total_example+=y.shape[0]
    return acc_sum/total_example,loss_sum

In [7]:
#定义模型训练函数
def train(model,train_loader,test_loader,loss,num_epochs,batch_size,params=None,lr=None,optimizer=None):
    train_ls = []
    test_ls = []
    for epoch in range(num_epochs): # 训练模型一共需要num_epochs个迭代周期
        train_loss_sum, train_acc_num,total_examples = 0.0,0.0,0
        for x, y in train_loader: # x和y分别是小批量样本的特征和标签
            y_pred = model(x)
            loss = loss_fn(y_pred, y)  #计算损失
            optimizer.zero_grad() # 梯度清零
            loss.backward()  # 反向传播
            optimizer.step() #梯度更新
            total_examples += y.shape[0]
            train_loss_sum += loss.item()
            train_acc_num += (y_pred.argmax(dim=1)==y).sum().item()
        train_ls.append(train_loss_sum)
        test_acc,test_loss = evaluate_testset(test_loader,model)
        test_ls.append(test_loss)
        print('epoch %d, train_loss %.6f,test_loss %f,train_acc %.6f,test_acc %.6f'%(epoch+1, train_ls[epoch],test_ls[epoch],train_acc_num/total_examples,test_acc))
    return 

In [8]:
num_epoch  = 20
batch_size = 64

In [ ]:
train(model,train_loader,test_loader,loss_fn,num_epoch,batch_size,params=model.parameters,lr=0.001,optimizer=optimizer)

epoch 1, train_loss 321.557572,test_loss 7.031791,train_acc 0.902517,test_acc 0.945500
epoch 2, train_loss 134.737983,test_loss 4.530779,train_acc 0.956817,test_acc 0.963900
epoch 3, train_loss 94.064229,test_loss 3.816852,train_acc 0.969617,test_acc 0.970000
epoch 4, train_loss 70.966020,test_loss 3.390277,train_acc 0.976150,test_acc 0.971100
epoch 5, train_loss 56.267231,test_loss 3.210675,train_acc 0.981400,test_acc 0.974300
epoch 6, train_loss 44.219717,test_loss 3.130155,train_acc 0.985533,test_acc 0.974600
epoch 7, train_loss 37.977134,test_loss 4.064653,train_acc 0.986833,test_acc 0.968400
epoch 8, train_loss 31.329893,test_loss 3.400227,train_acc 0.989317,test_acc 0.973800
epoch 9, train_loss 25.581067,test_loss 3.188283,train_acc 0.991350,test_acc 0.976400
epoch 10, train_loss 20.659922,test_loss 3.035384,train_acc 0.993033,test_acc 0.978300
epoch 11, train_loss 18.156700,test_loss 3.157790,train_acc 0.993550,test_acc 0.977800
epoch 12, train_loss 17.092509,test_loss 3.213999,